In [ ]:
import sys
sys.path.append("..")
import matplotlib
%matplotlib notebook
import matplotlib.pyplot as plt
import nidaqmx
import numpy as np
# from IPython.display import display, clear_output
# from skimage import data
# from skimage.feature import match_template

from LowLevelModules.NIdaq import *
from LowLevelModules.NIdaqAPD import *
from LowLevelModules.GeneralFunctions import *

#from collections import deque

class FSM:
    def __init__(self, x0=0, y0=0):
        
        self.position = {'x': x0, 'y': y0}    # this line will run setter function of position property
        self.conversion = {'x': 13.90, 'y': 9.69} # um/volt ???
    
#     @property
#     def position(self, unit='volts'):
#         return self._position
        
#     @position.setter
#     def position(self, new_pos=(0,0)):
#         # set the new current position
#         self._position = new_pos
    
    def micron_to_volts(self, pos_in_micron, axis):
        return pos_in_micron / self.conversion[axis]

    def volts_to_micron(self, pos_in_volts, axis):
        return pos_in_volts * self.conversion[axis]

    def read_position(self, unit='volts'):
        """ Voltage reading of the 2 AI from the DAQ
            Returns in Volts unless asked otherwise """
        with nidaqmx.Task() as fsm_task:
            fsm_task.ai_channels.add_ai_voltage_chan('/Dev1/ai1', 'FSM x axis')
            fsm_task.ai_channels.add_ai_voltage_chan('/Dev1/ai0', 'FSM y axis')
            curr_x, curr_y = fsm_task.read()
        self.position['x'], self.position['y'] = curr_x, curr_y
        if unit != 'volts':
            curr_x = self.volts_to_micron(curr_x, 'x')
            curr_y = self.volts_to_micron(curr_y, 'y')
        return curr_x, curr_y

    def calc_trajectory_in_V(self, final_pos, n_pts, axis):
        """  Returns an array in volts of a sine-like profile 
             for sweeping the values from one position to the next """
        init_pos = self.position[axis]
        delta_pos = self.micron_to_volts(new_y, axis) - init_pos
        x_sine = np.linspace(-np.pi/2, np.pi/2, n_pts)
        sweep_array = (np.sin(x_sine) + 1)/2*(final_pos-init_pos) + init_pos
        return sweep_array

    def go_to_position(self, x, y):
        """ Smooth sinusoidal move from current position to (x,y) in um """
        #curr_x, curr_y = read_position('microns')
        curr_x = self.volts_to_micron(self.position['x'])
        curr_y = self.volts_to_micron(self.position['y'])
        nx = int(np.ceil((abs(x-curr_x)*500+10)/2))
        ny = int(np.ceil((abs(y-curr_y)*500+10)/2))    
        n_max = max(nx,ny)

        # 2 arrays need to have the same size
        xVsweep_array = calc_trajectory_in_V(x, n_max, 'x')
        yVsweep_array = calc_trajectory_in_V(y, n_max, 'y')

        with nidaqmx.Task() as task:
            task.ao_channels.add_ao_voltage_chan( '/Dev1/ao0', 'FSM y axis', min_val=-9, max_val=9)
            task.ao_channels.add_ao_voltage_chan('/Dev1/ao1', 'FSM x axis', min_val=-9, max_val=9)
            task.timing.cfg_samp_clk_timing(rate= 50000, 
                                            active_edge=nidaqmx.constants.Edge.RISING ,
                                            sample_mode= nidaqmx.constants.AcquisitionType.FINITE, 
                                            samps_per_chan=len(xVsweep_array))
            task.write(np.array([yVsweep_array, xVsweep_array]), auto_start=True, timeout=5)
            task.wait_until_done(timeout=5)

        # set the new current position
        curr_x, curr_y = self.read_position()
        

    def scan_1D(self, x1, xf, mesh_pts=20, scan_rate=100, counter_terminal= "/Dev1/PFI0"):
        
        counterPort = "/Dev1/ctr1"
        mult_fac = 5

        ao_scan_rate = scan_rate*mult_fac
        ao_pts = mesh_pts*mult_fac

        # x trajectory 
        axis_scan = np.linspace(x1, xf, mesh_pts)

        # voltage scan is denser than the mesh    ---> HOW IS THAT POSSIBLE?
        v_ao_scan = micron_to_volts(np.linspace(x1, xf, ao_pts), 'x')

        # clock source
        counter_term_clk = '/Dev1/ctr0'
        trig_src =  '/Dev1/PFI12'       

        # setup counter
        countAPD = CI(counter_terminal, counterPort)
        countAPD.config_read_samples(trig_src, mesh_pts + 1, scan_rate)        

        # set up FSM AO AI
        FSMXtask = AO('/Dev1/ao1')
        FSMXtask.config_write(v_ao_scan, ao_scan_rate, trig_src)

        FSMreadXtask = AI('/Dev1/ai1')
        FSMreadXtask.config_read(ao_pts, ao_scan_rate, trig_src)

        # PFI12 start
        # CREATE EXT CLOCK TO GATE THE READING OF PULSES COMING FROM THE APD
        ext_clock_task = CO(counter_term_clk, scan_rate)

        # wait until done and read data
        FSMXtask.wait_until_done()
        FSMXtask.close()

        aiV = FSMreadXtask.read(number_of_samples_per_channel=ao_pts)
        thisX = volts_to_micron(np.asarray(aiV), axis)
        downSampleX = thisX[::mult_fac]
        FSMreadXtask.close()

        rawAPD=countAPD.read(number_of_samples_per_channel=mesh_pts+1)
        correctAPD = np.diff(rawAPD)*scan_rate/1000 # kHz

        countAPD.close()
        ext_clock_task.close()
        return downSampleX, correctAPD     


In [ ]:
# initialize the FSM object
FSM1 = FSM()
curr_x, curr_y = FSM1.position.values()

In [ ]:
#####################   USER INPUT   #####################
scan_size = 20    #um
mesh_size = 20    #um
scan_rate = 100   #Hz
##########################################################

curr_x, curr_y = FSM1.position.values()
start_x = curr_x - scan_size/2
end_x = curr_x + scan_size/2

start_y = curr_y + scan_size/2
end_y = curr_y - scan_size/2

#x_scan = np.linspace(start_x, end_x, mesh_size)
y_scan = np.linspace(start_y, end_y, mesh_size)

#FSM2D = np.zeros((mesh_x, mesh_y))
lp = LivePlot(1, 1, 5, 3, 'o', 'Coordinate (um)', 'APD counts (kHz)')

for idx, y0 in enumerate(y_scan):
    try:
        self.go_to_position(start_x, y0)
        xx, cts = scan_1D(start_x, end_x, mesh_size, scan_rate, 'x')
        #FSM2D[idx,:] = correctAPD
        lp.plot_live(xx, correctAPD)


    except KeyboardInterrupt:
        # press the stop button to trigger this
        FSMXtask.close()
        FSMreadXtask.close()
        countAPD.close()
        ext_clock_task.close()
        break
        
# back to initial location
go_to_position(curr_x, curr_y)        
